# 1. 导入必要的库

In [1]:
import pandas as pd
from langchain_community.llms import BaichuanLLM
import os
from langchain import LLMChain, PromptTemplate

# 设置Baichuan API Key
os.environ["BAICHUAN_API_KEY"] = "sk-f48e2e92eb606b7f67e63d905ba6492c"
llm = BaichuanLLM(model="Baichuan2-Turbo")

# 2. 创建Prompt模板和链

In [15]:
# 示例prompt模板
prompt_template = """
你是一个专业的产品经理，你需要根据评论内容进行分析，并给出对应的分析结果，你的分析结果必须严格按照固定格式来"好差评分析 - 一级属性 - 二级属性"，只需要提取最重要的一条就行，如果出现差评，优先提取差评的部分。
我有一个评论文本，需要根据情感、一级属性和二级属性进行分类和分析。
情感可以是“好评”，“中评”或“差评”。
一级属性包括“按键”，“包装”，“材质质感”，“操作”，“大小”，“功能”，“火力”，“加热”，“散热效果”，“烧水”，“外观”，“物流”，“颜值”，“质量”，“做工”，“噪音”，“重量”和“结构设计”。
二级属性可以是以下之一：“安全”，“按键不灵敏”，“按键灵敏”，“包装破损”，“不安全”，“不糊锅底”，“不可定时”，“不耐脏”，“不挑锅具”，“材质质感差”，“材质质感好”，“操作方便”，“操作麻烦”，“尺寸不合适”，“尺寸不适合”，“尺寸合适”，“尺寸适合”，“档位多”，“电源线设计问题”，“防辐射”，“防滑”，“防水”，“服务差”，“功能模式多”，“功能排序合理”，“锅薄”，“锅厚”，“厚度超薄”，“厚度较厚”，“糊锅底”，“火力不稳定”，“火力好调节”，“火力强”，“火力弱”，“火力稳定”，“火力易控制”，“加热不均匀”，“加热均匀”，“加热快”，“加热慢”，“加热面积大”，“加热面积小”，“可以定时”，“可以定温”，“没有异味”，“面板不清晰”，“面板大”，“面板破损”，“面板清晰”，“面板脏”，“磨砂面板”，“耐脏”，“能耗大”，“能耗小”，“轻”，“清洗方便”，“清洗困难”，“散热差”，“散热好”，“手动控制”，“塑料感强”，“提示不到位”，“提示到位”，“提手设计”，“挑锅具”，“童锁功能”，“斜屏设计”，“旋钮松”，“颜色好看”，“颜值高”，“易于携带”，“有异味”，“有预约”，“质量差”，“质量好”，“智能控制”，“重”，“做工粗糙”，“做工精细”，“炒菜的噪音大”，“炒菜的噪音小”，“电流异响声大”，“电流异响声小”，“提示蜂鸣声大”，“提示蜂鸣声小”，“廉价感”，“缝隙小”，“缝隙大”。如果评论中同时出现好评和差评，则在好差评分析中标为中评，并优先提取差评的属性。
二级属性可能不全，你可以根据一级属性来自己总结二级属性。
严格按照示例输出！！！

根据上述指南，分析评论并进行如下分类：

好差评分析： [好评/中评/差评]
一级属性： [一级属性]
二级属性： [二级属性]
示例输入：
评论文本：“产品颜值很高，质感也不错，但是塑料感太强了。”

示例输出：

"中评" - "材质质感" - "塑料感强"

以下是一些原始评论及其分析的案例：
{examples}
现在请分析以下评论：
原始评论：{comment}
"""

# 示例数据
examples = [
    ("颜值高，质感好，但是美中不足的是有点塑料感", "中评 - 材质质感 - 塑料感强"),
    ("按键不灵敏，包装破损", "差评 - 按键 - 按键不灵敏"),
    ("大小合适，功能多样，操作方便", "好评-功能-功能多样"),
    ("工作声音太大，不易清洁，外观倒漂亮，但工作人员服务太差，明明就是同一台才下单就有五元差价，还说不一样，这简直就是完美骗人。这个给小熊差评。","差评 - 噪音 - 工作声音大"),
    ("如果家庭做饭吃真心不建议买这款电陶炉，很慢很慢，可能家里烙饼能行，今天第一次炒菜，炒到你怀疑人生没有一丝烟火气，平时大蒜爆香下，这个火气不够，吃饭纯粹就是可能可以做熟吧，还耗时特别久。炒出来的味道也是一言难尽 火力如何：太低太低，煮茶喝都慢 ","差评 - 火力 - 火力太低")
]

# 生成示例部分的文本
example_texts = '\n'.join([f'原始评论：{example[0]} -> 分析：{example[1]}' for example in examples])

# 创建 PromptTemplate 对象
prompt = PromptTemplate(
    input_variables=["examples", "comment"],
    template=prompt_template
)

# 创建 LLMChain 对象
chain = LLMChain(prompt=prompt, llm=llm)


# 3.读取 Excel 文件并分析评论

In [16]:
%%time
# 读取 Excel 文件
df = pd.read_excel('c:\\Users\\admin\\Desktop\\苏泊尔测试.xlsx')

# 预处理评论数据
comments = df['原始评论'].tolist()

# 分析评论并获取结果
analysis_results = []
for comment in comments:
    result = chain.run(examples=example_texts, comment=comment).strip()
    print(f"原始评论: {comment}")
    print(f"分析结果: {result}")
    analysis_results.append(result)
    
# 解析分析结果
def parse_analysis_result(result):
    try:
        rating, primary_attr, secondary_attr = result.split('-')
        print(f"分析结果: 好差评分析-{rating}，一级属性-{primary_attr}，二级属性-{secondary_attr}")
    except ValueError:
        print("无法解析结果")
        rating, primary_attr, secondary_attr = "未知", "未知", "未知"
    return rating.strip(), primary_attr.strip(), secondary_attr.strip()

parsed_results = [parse_analysis_result(result) for result in analysis_results]

# 将结果写入 DataFrame
df['好差评分析'], df['一级属性'], df['二级属性'] = zip(*parsed_results)




原始评论: 德国大品牌，外观时尚大气，操作简便，居家必备。
分析结果: "好评" - "外观" - "时尚大气"
原始评论: 电陶炉收到了，质量非常好，做工精细，外观很漂亮，非常满意。
分析结果: "好评" - "质量" - "质量好"
原始评论: 电陶炉收到了，质量很好，外观大气，功能齐全，操作简单
分析结果: "好评" - "质量" - "质量好"
原始评论: 外观漂亮，加热速度挺快，我这个是3500瓦最的好，用来炒菜适宜，散热好，火力猛，太实用了，最可心的一次购物，客服也很有耐心给力，
分析结果: "好评" - "加热" - "加热快"
原始评论: 电陶炉尺寸标准，外观大气，功能齐全，操作简单很好用
分析结果: "好评" - "功能" - "功能齐全"
原始评论: 电磁炉款式新颖独特，使用起来很方便，而且外观设计简约大气上档次，家用功率很大，煲汤，火锅，炒菜的不二之选。
分析结果: "好评" - "外观" - "外观设计简约大气上档次"
原始评论: 电陶炉外观大气，功率大，烧水，炒菜都快速，按键灵敏，很满意的
分析结果: "好评" - "外观" - "大气"
原始评论: 电陶炉外观漂亮，简约大方，功能齐全，加热快
分析结果: "好评" - "外观" - "简约大方"
原始评论: 电陶炉外观很漂亮，是大功率的，升温速度很快，还不挑炉具，按键很灵敏，用着安全放心
分析结果: "好评" - "外观" - "漂亮"
原始评论: 火力很大，外观也好看，就是散热真的很慢😂
分析结果: "中评" - "火力" - "火力大"
"中评" - "散热效果" - "散热慢"
原始评论: 导热很快。刚开始用有一点点气味，可能用久了，应该就没有了，毕竟也是电气化的东西有气味很正常。开起来感受火力很猛，冬天可以用来烤火，取暖。刚打开还没有是砂锅，不知道快不快，但是单从感受的话应该是很快。3500一般家里电线不好的，建议别买，不然带不起来，但是也可以开到2000底部看起来不是很亮强迫症看起来有点儿难受，感觉底部做成镜面看起来更好看，钛金看起来很漂亮，毕竟家里的电磁炉也是黑的买一个不一样的，颜色会有一种新鲜感。 声音说大不大说小不小，陶炉本身不是很重没用，我期待的那么重单手提起来毫无压力，个人期望重一点点这样感觉比较固定，以上就是刚开始，想给我的感受。后续测评
分析结果: "中评" - "

In [17]:
# 保存结果到新的 Excel 文件
df.to_excel('c:\\Users\\admin\\Desktop\\刘浩宇_苏泊尔测试.xlsx', index=False)